In [1]:
%env JAVA_TOOL_OPTIONS='--add-opens=java.base/java.util=ALL-UNNAMED --add-opens=java.base/java.lang=ALL-UNNAMED'
%env FLINK_CONF_DIR=/Users/lgfquentin/dev/medical-dashboard/back-end-synthea-flink/config

env: JAVA_TOOL_OPTIONS='--add-opens=java.base/java.util=ALL-UNNAMED --add-opens=java.base/java.lang=ALL-UNNAMED'
env: FLINK_CONF_DIR=/Users/lgfquentin/dev/medical-dashboard/back-end-synthea-flink/config


In [2]:
%load_ext streaming_jupyter_integrations.magics

Could not find scli config file at '/Users/lgfquentin/dev/medical-dashboard/back-end-synthea-flink/.streaming_config.yml'. Will not load any secrets.
Set env variable JAVA_TOOL_OPTIONS='--add-opens=java.base/java.util=ALL-UNNAMED --add-opens=java.base/java.lang=ALL-UNNAMED'


In [3]:
%flink_connect --execution-target remote --remote-hostname localhost --remote-port 8081


remote environment has been created.


In [4]:
%%flink_execute_sql
CREATE TABLE patient (
  `id` STRING,
  `name` ROW(`family` STRING, `given` STRING ARRAY) ARRAY,
  `gender` STRING,
  `birthDate` STRING,
  `telecom` ROW ( `value` STRING ) ARRAY,
  `address` ROW (
    `line` STRING ARRAY,
    `city` STRING,
    `state` STRING,
    `postalCode` STRING,
    `country` STRING
  ) ARRAY,
  `maritalStatus` ROW (
    `text` STRING
  )
) WITH (
  'connector' = 'kafka',
  'topic' = 'Patient',
  'properties.bootstrap.servers' = 'kafka-edge1:9092',
  'properties.group.id' = 'testGroup',
  'scan.startup.mode' = 'earliest-offset',
  'format' = 'json'
)

Job starting...
Job started
Execution successful


In [5]:
%%flink_execute_sql
CREATE TABLE patient_psql (
  `identifier` STRING,
  `surname` STRING,
  `familyName` STRING,
  `gender` STRING,
  `birthDate` STRING,
  `telecom` STRING,
  `adress` STRING,
  `city` STRING,
  `state` STRING,
  `postalCode` STRING,
  `country` STRING,
  `maritalStatus` STRING,
  PRIMARY KEY (`identifier`) NOT ENFORCED
) WITH (
  'connector' = 'jdbc',
  'url' = 'jdbc:postgresql://db:5432/example',
  'table-name' = 'public.patient',
  'username' = 'example',
  'password' = 'example'
);


Job starting...
Job started
Execution successful


In [6]:
%%flink_execute_sql
INSERT INTO patient_psql
SELECT id,
 p.name[1].family as familyname,
  p.name[1].given[1] as surname,
   gender,
    birthDate,
     p.telecom[1][1] as telecom,
      p.address[1].line[1] as address,
        p.address[1].city as city,
         p.address[1].state as state,
          p.address[1].country as country,
           p.address[1].postalCode as postalCode,
            maritalStatus.text as maritalStatus from patient p;

This job runs in a background, please either wait or interrupt its execution before continuing


Job starting...
Job started
